### 🪄 Install `raga-testing-platform` library

In [ ]:
pip install raga-testing-platform==1.0.36

### Put your `ACCESS KEY & SECRET KEY`

In [ ]:
# Define the access key, secret key, and host
ACCESS_KEY = "nnXvot82D3idpraRtCjJ"
SECRET_KEY = "P2doycL4WBZXLNARIs4bESxttzF3MHSC5K15Jrs9"
HOST = "http://65.0.13.122:8080/"

# Define the configuration directory path
CONFIG_DIR = "/root/.raga" #COLAB
# CONFIG_DIR = "~/.raga" # LOCAL MACHINE

### Create .raga dir for raga config

In [ ]:
!mkdir $CONFIG_DIR

### Create config file and put configs

In [ ]:
!touch $CONFIG_DIR/config
!echo -e "[default]\nraga_access_key_id=$ACCESS_KEY\nraga_secret_access_key=$SECRET_KEY\napi_host=$HOST" > $CONFIG_DIR/config


### Verify Config file values

In [ ]:
!cat $CONFIG_DIR/config

### Import All raga lib from raga module

In [ ]:
from raga import *

### 👟 Run an experiment
1️⃣. Update `project_name` name and `run_name`.

2️⃣. Run `TestSession` object


In [ ]:
project_name = "testingProject"
run_name = "colab-8-aug-v4"
test_session = TestSession(project_name=project_name,run_name=run_name)

### 👟 Run an experiment on AB Labelled Test
You can update the `test_name` & `dataset_name`

In [ ]:
dataset_name="dataset-7-aug-v3"
test_name="AB-test-8-aug-v4"

rules = ModelABTestRules() 
rules.add(metric="precision_diff_all", IoU=0.5, _class="All", threshold=0.5)

model_comparison_check = model_ab_test(test_session=test_session, 
                                       dataset_name=dataset_name,
                                       test_name=test_name,
                                       modelA = "modelA", 
                                       modelB = "modelB" ,
                                       type = "labelled", 
                                       gt="GT",
                                       rules = rules,
                                       aggregation_level=["location", "vehicle_no", "date_captured", "channel", "dataset_type"])
# add payload into test_session object
test_session.add(model_comparison_check)
#run added ab test model payload
test_session.run()

### 👟 Run an experiment on AB Unlabelled Test
You can update the `test_name` & `dataset_name`

In [ ]:
dataset_name="dataset-7-aug-v3"
test_name="AB-test-8-aug-v5"

rules = ModelABTestRules() 
rules.add(metric = "difference_all", IoU = 0.5, _class = "ALL", threshold = 0.5)

model_comparison_check = model_ab_test(test_session=test_session, 
                                       dataset_name=dataset_name,
                                       test_name=test_name,
                                       modelA = "modelA", 
                                       modelB = "modelB" ,
                                       type = "unlabelled", 
                                       rules = rules,
                                       aggregation_level=["location", "vehicle_no", "date_captured", "channel", "dataset_type"])

# add payload into test_session object
test_session.add(model_comparison_check)
#run added test
test_session.run()

### 👟 Run an experiment on Image Drift Detection
You can update the `test_name` & `train_dataset_name`, `field_dataset_name`

In [ ]:
test_name="AB-test-8-aug-v1"
train_dataset_name="train-dataset-7-aug-v2"
field_dataset_name="field-dataset-7-aug-v2"

rules = DriftDetectionRules()
rules.add(type="anomaly_detection", dist_metric="Euclidian", _class="All", threshold=0.6)

edge_case_detection = data_drift_detection(test_session=test_session,
                                           test_name=test_name,
                                           train_dataset_name=train_dataset_name,
                                           field_dataset_name=field_dataset_name,
                                           train_embed_col_name="ImageEmbedding",
                                           field_embed_col_name = "ImageEmbedding" ,
                                           level = "image",
                                           aggregation_level=["location", "vehicle_no", "date_captured", "channel", "dataset_type"],
                                           rules = rules)

# add payload into test_session object
test_session.add(edge_case_detection)
#run added test
test_session.run()